In [ ]:
# Step 1: Install Necessary Libraries
!pip install -q pandas requests
#!pip uninstall markdown -y
!pip install markdown


In [ ]:
# Step 2: Import Libraries
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import markdown
import os


In [ ]:
import os
env_vars = !cat .env
for var in env_vars:
    key, value = var.split('=')
    print(key)
    #print(value)
    os.environ[key] = value

In [ ]:
# Step 3: Load Data
# Read report.txt which is a markdown file
with open('report.txt', 'r') as file:
    report_content = file.read()

# Step 4: Load SEO Keywords
# Read SEO keywords from 'seo_report.md'
with open('seo_report.md', 'r') as file:
    seo_content = file.read()

In [ ]:
# Extract keywords assuming they are listed after 'SEO Keywords:'
import re
keywords_match = re.search(r'SEO Keywords:\s*(.*)', seo_content)
if keywords_match:
    seo_keywords = keywords_match.group(1)
else:
    seo_keywords = ""

print(seo_keywords)


In [ ]:
# Step 5: Load CSV Data
# Read output.csv which contains the recommendations in a tabular format
output_data = pd.read_csv('output.csv')

# Convert the CSV data to an HTML table
output_html_table = output_data.to_html(index=False)
print(output_html_table)


In [ ]:
# Step 6: Process Data
# Add SEO keywords to the post content
report_content += f"\n\n#### SEO Keywords:\n{seo_keywords}"

# Convert Markdown to HTML
post_content_html = markdown.markdown(report_content)

# Append the HTML table to the post content
post_content_html += f"<h3>Forex Trade Recommendations Table</h3>\n{output_html_table}"


In [ ]:
# Step 7: Generate WordPress Post
# Define your WordPress site and API credentials
wordpress_site = "https://" + str(os.getenv('WORDPRESS_URL'))
api_endpoint = f"{wordpress_site}/wp-json/wp/v2/posts"
username = os.getenv('WORDPRESS_USERNAME')
print(username)
password = os.getenv('WORDPRESS_PASSWORD')
print(password)

# Category ID (replace with your actual category ID)
category_id = 17

# Create the post data
post_data = {
    'title': 'Forex Trade Recommendations for 2024-06-29 19:10 UTC',
    'content': post_content_html,
    'status': 'publish',  # or 'draft' if you want to review before publishing
    'categories': [category_id],  # Add category ID(s) here
    'meta': {
        'seo_keywords': seo_keywords  # Custom meta field for SEO keywords
    }
}

In [ ]:
# Make the API request
response = requests.post(api_endpoint, json=post_data, auth=HTTPBasicAuth(username, password))

# Check the response
if response.status_code == 201:
    print("Post created successfully!")
else:
    print("Failed to create post:", response.status_code, response.text)